In [15]:
import pandas as pd
import sqlite3
import psycopg2
from dotenv import load_dotenv
from psycopg2 import extras
import os
load_dotenv()

True

### Load Data

Im only loading in 10,000 rows because our heroku account can only hold 10,000 rows. Our production implementation would house all the data

In [2]:
# Data from EPA includes all vehicles from ~1980-2020, choose your appropriate path
path = "../data/vehicles.csv"
# path = "https://raw.githubusercontent.com/mpHarm88/streetsmart/master/data/vehicles.csv"

df = pd.read_csv(path, nrows=10000)
print(f"The shape of the data: {df.shape}")

The shape of the data: (10000, 83)


### SQLite3

SQL Refresher: https://colab.research.google.com/drive/1TBEFJvxTwkIKf0HaXAsvjeB61RHhy2Lq

Make sure that you dont already have a sqlite3 database called "epa_data_test.sqlite3", if you already do then this cell will error. Delete the database in your working directory and execute the cell again.

In [3]:
# Create connection to local sqlite3 database
sl_conn = sqlite3.Connection("epa_data_test.sqlite3")
sl_curs = sl_conn.cursor()

In [68]:
# Populate the database. The table name is "epa_vehicles_test" and verify in TablePlus

# index=False so we dont create another column, instead we will create the already represented id column as the serial primary key.
df.to_sql("epa_vehicles_test", sl_conn, index=False)

In [4]:
# Verify by counting how many rows are in the data
all_data = sl_curs.execute("SELECT COUNT(*) FROM epa_vehicles_test").fetchall()

# all_data contains 10,000 rows as expected
all_data

[(10000,)]

### Psycopg2 and Heroku

In [26]:
#Load credentials from .env
name = os.getenv("DB_NAME_AWS")
password = os.getenv("DB_PW_AWS")
host = os.getenv("DB_HOST_AWS")
user = os.getenv("DB_USER_AWS")
port = os.getenv("DB_PORT_AWS")


# Create connection to heroku database
pg_conn = psycopg2.connect(dbname="database-1",
                           user="postgres",
                           password="awslabs24",
                           host="database-1.citfaabbbnpe.us-east-1.rds.amazonaws.com"
                          )

# Create cursor object
pg_curs = pg_conn.cursor()

OperationalError: FATAL:  database "database-1" does not exist


'streetsmarts-epa'

### Create Table Statement

I think we should keep the original table names, this makes it easier for the next LABS group to recognize each column and keep continuity with the data dictionary provided by the EPA

In [33]:
#Look at column names and dtypes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 83 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   barrels08        10000 non-null  float64
 1   barrelsA08       10000 non-null  float64
 2   charge120        10000 non-null  float64
 3   charge240        10000 non-null  float64
 4   city08           10000 non-null  int64  
 5   city08U          10000 non-null  float64
 6   cityA08          10000 non-null  int64  
 7   cityA08U         10000 non-null  float64
 8   cityCD           10000 non-null  float64
 9   cityE            10000 non-null  float64
 10  cityUF           10000 non-null  float64
 11  co2              10000 non-null  int64  
 12  co2A             10000 non-null  int64  
 13  co2TailpipeAGpm  10000 non-null  float64
 14  co2TailpipeGpm   10000 non-null  float64
 15  comb08           10000 non-null  int64  
 16  comb08U          10000 non-null  float64
 17  combA08      

In [92]:
table = """
CREATE TABLE epa_vehicles_all(
    barrels08 REAL,
    barrelsA08 REAL,
    charge120 REAL,
    charge240 REAL,
    city08 REAL,
    city08U REAL,
    cityA08 REAL,
    cityA08U REAL,
    cityCD REAL,
    cityE REAL,
    cityUF REAL,
    co2 REAL,
    co2A REAL,
    co2TailpipeAGpm REAL,
    co2TailpipeGpm REAL,
    comb08 REAL,
    comb08U REAL,
    combA08 REAL,
    combA08U REAL,
    combE REAL,
    combinedCD REAL,
    combinedUF REAL,
    cylinders REAL,
    displ REAL,
    drive VARCHAR,
    engID REAL,
    eng_dscr VARCHAR,
    feScore REAL,
    fuelCost08 REAL,
    fuelCostA08 REAL,
    fuelType VARCHAR,
    fuelType1 VARCHAR,
    ghgScore REAL,
    ghgScoreA REAL,
    highway08 REAL,
    highway08U REAL,
    highwayA08 REAL,
    highwayA08U REAL,
    highwayCD REAL,
    highwayE REAL,
    highwayUF REAL,
    hlv REAL,
    hpv REAL,
    id SERIAL PRIMARY KEY,
    lv2 REAL,
    lv4 REAL,
    make VARCHAR,
    model VARCHAR,
    mpgData VARCHAR,
    phevBlended REAL,
    pv2 REAL,
    pv4 REAL,
    range REAL,
    rangeCity REAL,
    rangeCityA REAL,
    rangeHwy REAL,
    rangeHwyA REAL,
    trany VARCHAR,
    UCity REAL,
    UCityA REAL,
    UHighway REAL,
    UHighwayA REAL,
    VClass VARCHAR,
    year REAL,
    youSaveSpend REAL,
    guzzler VARCHAR,
    trans_dscr VARCHAR,
    tCharger VARCHAR,
    sCharger VARCHAR,
    atvType VARCHAR,
    fuelType2 VARCHAR,
    rangeA VARCHAR,
    evMotor VARCHAR,
    mfrCode VARCHAR,
    c240Dscr VARCHAR,
    charge240b REAL,
    c240bDscr VARCHAR,
    createdOn VARCHAR,
    modifiedOn VARCHAR,
    startStop VARCHAR,
    phevCity REAL,
    phevHwy REAL,
    phevComb REAL
)
"""

### Only run the next cell if you want t drop the table from the database

In [93]:
# pg_curs.execute("DROP TABLE epa_vehicles_all")
# pg_conn.commit()

In [94]:
# Create the table
pg_curs.execute(table)

# Save the results
pg_conn.commit()

### Insert the Data

Stack Overflow Help: ## h/t: https://stackoverflow.com/questions/8134602/psycopg2-insert-multiple-rows-with-one-query

Execute Values Docs: https://www.psycopg.org/docs/extras.html#psycopg2.extras.execute_values

In [85]:
# Gathering all the data
data = sl_curs.execute("SELECT * FROM epa_vehicles_test").fetchall()

# Length of values matches the length of columns
len(data[0])

83

In [95]:
# Set up the query
query = """INSERT INTO epa_vehicles_all (barrels08 ,barrelsA08,charge120,charge240,city08,city08U,cityA08,cityA08U,cityCD,cityE,cityUF,
co2,co2A,co2TailpipeAGpm,co2TailpipeGpm,comb08,comb08U,combA08,combA08U,combE,combinedCD,combinedUF,cylinders,displ,drive,engID,eng_dscr,
feScore,fuelCost08,fuelCostA08,fuelType,fuelType1,ghgScore,ghgScoreA,highway08,highway08U,highwayA08,highwayA08U,highwayCD,highwayE,
highwayUF,hlv,hpv,id,lv2,lv4,make,model,mpgData,phevBlended,pv2,pv4,range,rangeCity,rangeCityA,rangeHwy,rangeHwyA,trany,UCity,UCityA,UHighway,
UHighwayA,VClass,year,youSaveSpend,guzzler,trans_dscr,tCharger,sCharger,atvType,fuelType2,rangeA,evMotor,mfrCode,c240Dscr,charge240b,
c240bDscr,createdOn,modifiedOn,startStop,phevCity,phevHwy,phevComb) VALUES %s"""

# Insert the data
extras.execute_values(pg_curs, query, data)

# Save the results
pg_conn.commit()

### Test

In [19]:
# Test to see if its possible to make a query to the database
pg_curs.execute("SELECT * FROM epa_vehicles_all;")
pg_curs.fetchall()[0]

(15.695714,
 0.0,
 0.0,
 0.0,
 19.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 -1.0,
 -1.0,
 0.0,
 423.1905,
 21.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 4.0,
 2.0,
 'Rear-Wheel Drive',
 9011.0,
 '(FFS)',
 -1.0,
 1300.0,
 0.0,
 'Regular',
 'Regular Gasoline',
 -1.0,
 -1.0,
 25.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1,
 0.0,
 0.0,
 'Alfa Romeo',
 'Spider Veloce 2000',
 'Y',
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 'Manual 5-spd',
 23.3333,
 0.0,
 35.0,
 0.0,
 'Two Seaters',
 1985.0,
 -1500.0,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 0.0,
 None,
 'Tue Jan 01 00:00:00 EST 2013',
 'Tue Jan 01 00:00:00 EST 2013',
 None,
 0.0,
 0.0,
 0.0)

### Close all connections and cursors

If you dont close the connection it will remain open, heroku only allows a few connections to be open to the database at a time, please close them.

In [ ]:
# SQLite3
sl_curs.close()
sl_conn.close()

In [22]:
# Psycopg2
pg_conn.close()
pg_curs.close()